In [96]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
import matplotlib.pyplot as plt

In [58]:
# uploading the data file
df = pd.read_csv('Documents/UW_Courses/511_DataViz/C4/SDOT_CollisionsData_Processesd_S1_28Nov2024.csv')

In [60]:
# Initial data exploration      
df.head(5)

,INCKEY,LOCATION,INCDATE,Year,INCDTTM,WEATHER,ROADCOND,LIGHTCOND,x,y
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,1/10/2015 1:23:00 AM,Overcast,Wet,Dark - Street Lights On,1.267051e+06,245201.683375
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,1/8/2015 8:43:00 AM,Clear,Dry,Daylight,1.260492e+06,196498.281853
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,2/6/2016 4:31:00 PM,Overcast,Dry,Daylight,1.261960e+06,195155.225276
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,1/15/2016 3:45:00 PM,Overcast,Wet,Dusk,1.273579e+06,229455.369055
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,1/27/2016 9:50:00 AM,Overcast,Dry,Daylight,1.266318e+06,241815.895852


In [62]:
df.shape

(99259, 10)

In [72]:
# creating column for the time of accident
df["INCDATEnTIME"] = pd.to_datetime(df["INCDTTM"], errors = 'coerce')
for idx, dt in enumerate(df["INCDTTM"]):
    if ':' not in dt:
        df.loc[idx, "INCDATEnTIME"] = np.nan


In [74]:
df.head(5)

,INCKEY,LOCATION,INCDATE,Year,INCDTTM,WEATHER,ROADCOND,LIGHTCOND,x,y,INCDATEnTIME
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,1/10/2015 1:23:00 AM,Overcast,Wet,Dark - Street Lights On,1.267051e+06,245201.683375,2015-01-10 01:23:00
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,1/8/2015 8:43:00 AM,Clear,Dry,Daylight,1.260492e+06,196498.281853,2015-01-08 08:43:00
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,2/6/2016 4:31:00 PM,Overcast,Dry,Daylight,1.261960e+06,195155.225276,2016-02-06 16:31:00
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,1/15/2016 3:45:00 PM,Overcast,Wet,Dusk,1.273579e+06,229455.369055,2016-01-15 15:45:00
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,1/27/2016 9:50:00 AM,Overcast,Dry,Daylight,1.266318e+06,241815.895852,2016-01-27 09:50:00


In [76]:
# Unique value and column name
unique_value = 261597  # Replace with the actual value
column_name = "INCKEY"  # Replace with the column name containing unique values

# Filter the row without altering the original DataFrame
row = df[df[column_name] == unique_value]

# Print the row
print(row)


       INCKEY                                           LOCATION     INCDATE  \
61443  261597  M L KING JR WAY BETWEEN E SPRING ST AND E UNIO...  30/11/2016   

       Year     INCDTTM WEATHER ROADCOND LIGHTCOND             x  \
61443  2016  11/30/2016     NaN      NaN       NaN  1.279608e+06   

                   y INCDATEnTIME  
61443  226770.240668          NaT  


In [87]:
df_1 = df.drop(['INCDTTM'], axis=1)

In [89]:
df_2 = df_1.dropna(how='any')

In [91]:
df_2.shape

(77667, 10)

In [94]:
df_2.head(5)

,INCKEY,LOCATION,INCDATE,Year,WEATHER,ROADCOND,LIGHTCOND,x,y,INCDATEnTIME
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,Overcast,Wet,Dark - Street Lights On,1.267051e+06,245201.683375,2015-01-10 01:23:00
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,Clear,Dry,Daylight,1.260492e+06,196498.281853,2015-01-08 08:43:00
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,Overcast,Dry,Daylight,1.261960e+06,195155.225276,2016-02-06 16:31:00
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,Overcast,Wet,Dusk,1.273579e+06,229455.369055,2016-01-15 15:45:00
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,Overcast,Dry,Daylight,1.266318e+06,241815.895852,2016-01-27 09:50:00


In [98]:
# Define the source CRS (State Plane Washington North, EPSG: 2285) and target CRS (WGS84, EPSG: 4326)
state_plane = Proj("EPSG:2285")  # Source CRS (State Plane, feet)
wgs84 = Proj("EPSG:4326")        # Target CRS (Latitude/Longitude, WGS84)

# Function to convert State Plane to Latitude/Longitude
def convert_state_plane_to_lat_lon(row):
    # The function only uses the 'x' and 'y' columns from the row
    lat, lon = transform(state_plane, wgs84, row['x'], row['y'])
    return pd.Series({'Latitude': lat, 'Longitude': lon})

# Apply the conversion to each row of the DataFrame
df_2[['Latitude', 'Longitude']] = df_2.apply(convert_state_plane_to_lat_lon, axis=1)
df_2.head(5)

/var/folders/yn/1bdhc0m15tj5q6jczyp818wc0000gn/T/ipykernel_53323/1584083632.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lat, lon = transform(state_plane, wgs84, row['x'], row['y'])
/var/folders/yn/1bdhc0m15tj5q6jczyp818wc0000gn/T/ipykernel_53323/1584083632.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[['Latitude', 'Longitude']] = df_2.apply(convert_state_plane_to_lat_lon, axis=1)
/var/folders/yn/1bdhc0m15tj5q6jczyp818wc0000gn/T/ipykernel_53323/1584083632.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,INCKEY,LOCATION,INCDATE,Year,WEATHER,ROADCOND,LIGHTCOND,x,y,INCDATEnTIME,Latitude,Longitude
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,Overcast,Wet,Dark - Street Lights On,1.267051e+06,245201.683375,2015-01-10 01:23:00,47.662133,-122.348583
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,Clear,Dry,Daylight,1.260492e+06,196498.281853,2015-01-08 08:43:00,47.528283,-122.371246
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,Overcast,Dry,Daylight,1.261960e+06,195155.225276,2016-02-06 16:31:00,47.524682,-122.365199
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,Overcast,Wet,Dusk,1.273579e+06,229455.369055,2016-01-15 15:45:00,47.619322,-122.320864
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,Overcast,Dry,Daylight,1.266318e+06,241815.895852,2016-01-27 09:50:00,47.652813,-122.351288


In [100]:
df_2.head(5)

,INCKEY,LOCATION,INCDATE,Year,WEATHER,ROADCOND,LIGHTCOND,x,y,INCDATEnTIME,Latitude,Longitude
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,Overcast,Wet,Dark - Street Lights On,1.267051e+06,245201.683375,2015-01-10 01:23:00,47.662133,-122.348583
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,Clear,Dry,Daylight,1.260492e+06,196498.281853,2015-01-08 08:43:00,47.528283,-122.371246
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,Overcast,Dry,Daylight,1.261960e+06,195155.225276,2016-02-06 16:31:00,47.524682,-122.365199
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,Overcast,Wet,Dusk,1.273579e+06,229455.369055,2016-01-15 15:45:00,47.619322,-122.320864
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,Overcast,Dry,Daylight,1.266318e+06,241815.895852,2016-01-27 09:50:00,47.652813,-122.351288


In [110]:
df_3 = df_2.drop(['x','y'], axis=1)
df_3 = df_3.rename(columns={'INCDATEnTIME': 'INCDTTM'})

In [112]:
df_3.shape

(77667, 10)

In [114]:
df_3.head(10)

,INCKEY,LOCATION,INCDATE,Year,WEATHER,ROADCOND,LIGHTCOND,INCDTTM,Latitude,Longitude
0,182923,LINDEN AVE N AND N 46TH ST,10/01/2015,2015,Overcast,Wet,Dark - Street Lights On,2015-01-10 01:23:00,47.662133,-122.348583
1,182921,30TH AVE SW AND SW THISTLE ST,08/01/2015,2015,Clear,Dry,Daylight,2015-01-08 08:43:00,47.528283,-122.371246
2,213535,SW TRENTON ST BETWEEN 25TH AVE SW AND 26TH AVE SW,06/02/2016,2016,Overcast,Dry,Daylight,2016-02-06 16:31:00,47.524682,-122.365199
3,212581,BROADWAY E BETWEEN E DENNY WAY AND E OLIVE WAY,15/01/2016,2016,Overcast,Wet,Dusk,2016-01-15 15:45:00,47.619322,-122.320864
4,212387,EVANSTON AVE N BETWEEN N 36TH ST AND N 39TH ST,27/01/2016,2016,Overcast,Dry,Daylight,2016-01-27 09:50:00,47.652813,-122.351288
5,212938,WHALLEY PL W BETWEEN W DRAVUS ST AND W BERTONA ST,04/02/2016,2016,Raining,Wet,Dark - Street Lights On,2016-02-04 00:49:00,47.649496,-122.390826
6,211717,S GRAHAM ST BETWEEN SWIFT AVE S AND 21ST AVE S,11/01/2016,2016,Raining,Wet,Daylight,2016-01-11 14:45:00,47.546950,-122.306891
8,212959,FAIRVIEW AVE N AND MERCER S SR ST,04/02/2016,2016,Clear,Dry,Daylight,2016-02-04 11:41:00,47.624265,-122.334285
13,207741,2ND AV ET S AND S WASHINGTON ST,27/11/2015,2015,Clear,Dry,Daylight,2015-11-27 07:53:00,47.600878,-122.331215
14,212586,RENTON AVE S BETWEEN S BARTON ST AND S FLETCHE...,31/01/2016,2016,Clear,Dry,Dark - Street Lights On,2016-01-31 03:28:00,47.520143,-122.272546


In [116]:
# Output file path (directly using your given output path)
output_file_path = "Documents/UW_Courses/511_DataViz/C4/SDOT_CollisionsData_Processesd_S2_28Nov2024.csv"

In [118]:
# Save the DataFrame to CSV
df_3.to_csv(output_file_path, index=False)